# Practica 4

np.multiply hace multiplicacion elemento a elemento. Para hacer esto, no usar dot ni matmul

In [1]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import checkNNGradients as checkGradiente

In [2]:
weights = loadmat('ex4weights.mat')
theta1, theta2 = weights['Theta1'], weights['Theta2']

data = loadmat('ex4data1.mat')
y = data['y']
X = data['X']

In [3]:
y = np.ravel(y)
m=len(y)
X_1 = np.hstack([np.ones([m, 1]), X])

In [4]:
def sigmoide(z):
    return (1.0/(1.0+np.exp(-z)))

In [5]:
def forwardprop(X, theta1, theta2):
    X_1 = np.hstack([np.ones([len(X), 1]), X])
    a_1 = X_1
    z_2 = np.matmul(a_1, theta1.T)
    a_2 = sigmoide(z_2)
    a_2 = np.hstack([np.ones([len(a_2), 1]), a_2])
    z_3 = np.matmul(a_2, theta2.T)
    a_3 = sigmoide(z_3)
    return a_1, z_2, a_2, z_3, a_3

In [6]:
a_1, z_2, a_2, z_3, a_3 = forwardprop(X, theta1, theta2)

In [7]:
print(a_3.shape)

(5000, 10)


In [8]:
def yOneHot(y, num_etiquetas):
    y = (y-1)
    y_onehot = np.zeros((m, num_etiquetas)) #Matriz 5000 x 10
    for i in range(m):
        y_onehot[i][y[i]] = 1
    return y_onehot

In [9]:
def coste(H, y, m):
    Term1 = y * np.log(H) 
    Term2 =  (1 - y) * np.log(1 - H)
    cost = ( 1 / m) * np.sum(-Term1 - Term2)
    return cost

In [10]:
y_ = yOneHot(y, 10)
cost = coste(a_3, y_,m)
print(cost)

0.2876291651613189


In [11]:
def coste_reg(theta1, theta2, coste, lamb, m):
    reg = (np.sum(np.power(theta1[:, 1:], 2)) + np.sum(np.power(theta2[:, 1:], 2))) 
    coste_reg = coste + (reg* (lamb/(2*m)))
    return coste_reg

In [12]:
coste_reg = coste_reg(theta1, theta2, cost, 1, m)
print(coste_reg)

0.38376985909092365


In [13]:
def func_g(z):
    ones = np.ones(z.shape)
    a = sigmoide(z)
    return a*(ones-a)

In [14]:
def backprop(params_rn, num_entradas, num_ocultas, num_etiquetas, X, y, reg):
    Theta1 = np.reshape(params_rn[:num_ocultas * (num_entradas + 1)],(num_ocultas, (num_entradas + 1)))
    Theta2 = np.reshape(params_rn[num_ocultas * (num_entradas + 1):],(num_etiquetas, (num_ocultas + 1)))
    
    #Forward propagation
    a1, z2, a2, z3, a3 = forwardprop(X, Theta1, Theta2)
    
    #Backward propagation
    sigma_3 = np.array(a3 - y)
    sigma_2 = np.matmul(Theta2.T[1:, :], sigma_3.T) * func_g(z2).T
    
    delta_1 = np.zeros(Theta1.shape)
    delta_2 = np.zeros(Theta2.shape)
    
    delta_1 = delta_1 + np.matmul(sigma_2, a1)
    delta_2 = delta_2 + np.matmul(sigma_3.T, a2)
    
    grad_1 = (1/len(y))*(delta_1) + (reg/len(y))*np.r_[[np.zeros(Theta1.shape[1])],Theta1[1:]]
    grad_2 = (1/len(y))*(delta_2) + (reg/len(y))*np.r_[[np.zeros(Theta2.shape[1])],Theta2[1:]]
    
    gradiente = np.concatenate((grad_1, grad_2), axis = None)
    
    cost = coste(a3, y, len(y))
    
    return cost, gradiente

In [15]:
params_rn =  np.concatenate((theta1, theta2), axis = None)

In [16]:
print(backprop(params_rn, 400, 25, 10, X, y_, 0))

(0.2876291651613189, array([ 6.18712766e-05,  0.00000000e+00,  0.00000000e+00, ...,
        9.66104721e-05, -7.57736846e-04,  7.73329872e-04]))


In [17]:
print(checkGradiente.checkNNGradients(backprop, 0))

grad shape:  (38,)
num grad shape:  (38,)
[ 5.27761307e-11 -3.29853332e-13  7.89324855e-12  9.17629688e-12
 -6.30465125e-11  4.30501815e-12 -1.07556533e-11 -4.15864704e-11
 -9.52194434e-11  9.26888080e-12 -4.42525738e-11 -1.26826244e-10
 -2.17855178e-11  5.45035371e-13 -1.04682513e-11 -2.27557070e-11
  1.93531996e-11 -4.96176017e-13  9.77740458e-12  2.73879461e-11
  6.03760375e-11  1.32927280e-11  6.81166235e-12  5.26766131e-12
  1.67883762e-11  1.88441207e-11  6.93309576e-11  1.56080426e-11
  4.89147611e-12  1.37491268e-11  1.70987668e-11  1.79336823e-11
  7.77324871e-11  1.60134683e-11  6.39786835e-12  1.78092541e-11
  1.66117953e-11  1.82341919e-11]


In [18]:
def inicializacionPesos(L_in, L_out):
    c_ini = 0.12
    peso = np.random.uniform(-c_ini, c_ini, size = (L_in, L_out))
    peso = np.insert(peso,0,1,axis = 0)
    return peso

In [19]:
def aprendizaje(num_entradas, num_ocultas, num_etiquetas, X, Y, reg, it):
    theta1 = inicializacionPesos(num_entradas, num_ocultas)
    theta2 = inicializacionPesos(num_ocultas, num_etiquetas)
    
    y_1 = yOneHot(Y, 10)
    
    params = np.hstack((np.ravel(theta1), np.ravel(theta2)))
    result = opt.minimize(fun = backprop, x0 = params, args = (num_entradas, num_ocultas, num_etiquetas, X, y_1, reg), method='TNC', jac = True, options = {'maxiter': it})

    Thetas1 = result.x[:(num_ocultas*(num_entradas+1))].reshape(num_ocultas,(num_entradas+1))
    Thetas2 = result.x[(num_ocultas*(num_entradas+1)):].reshape(num_etiquetas,(num_ocultas+1))

    h = forwardprop(X, Thetas1, Thetas2)[4]

    predicciones = h.argmax(axis = 1)

    y_ = Y-1
    
    comparacion = (predicciones == y_)*1

    precision = (sum(comparacion) / len(comparacion))*100

    print('La precisión es {} % '.format(precision))

In [40]:
aprendizaje(400, 25, 10, X, y, 1.5, 100)

La precisión es 94.62 % 
